In [1]:
import pandas as pd
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import logging
import sys
import re
import matplotlib.pyplot as plt
import io
import warnings
import json

In [2]:
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
%matplotlib inline 

In [3]:
class JupyterHandler(logging.Handler):
    """Обробник логів для виводу в консоль Jupyter."""
    def emit(self, record):
        time_str = "??:??:??"
        if self.formatter:
             time_str = self.formatter.formatTime(record, datefmt=self.formatter.datefmt) 
        color = {'INFO': '32', 'WARNING': '33', 'ERROR': '31', 'CRITICAL': '41'}.get(record.levelname, '0')
        msg = f"\033[{color}m{record.levelname}\033[0m: {record.getMessage()}"
        print(f"{time_str} | {msg}", file=sys.stdout)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = JupyterHandler()
    formatter = logging.Formatter(fmt='%(asctime)s', datefmt='%H:%M:%S')
    handler.setFormatter(formatter) 
    logger.addHandler(handler)

class DataCleanerApp:

    DATE_ELEMENTS = {'Рік (YYYY)': '%Y', 'Рік (YY)': '%y', 'Місяць (MM)': '%m', 'Місяць (Назва)': '%b', 'День (DD)': '%d', 'Година (HH)': '%H', 'Хвилина (MM)': '%M', 'Секунда (SS)': '%S', 'Мікросекунди (f)': '%f', '--- Вибрати ---': ''}
    DATE_SEPARATORS = {'- (Тире)': '-', '/ (Слеш)': '/', '. (Крапка)': '.', ', (Кома)': ',', 'Пробіл (Space)': ' ', ': (Двокрапка)': ':', '--- Вибрати ---': ''}
    
    CYR_TO_LAT_FULL = {'А': 'A', 'Б': 'B', 'В': 'V', 'Г': 'H', 'Ґ': 'G', 'Д': 'D', 'Е': 'E', 'Є': 'Ye', 'Ж': 'Zh', 'З': 'Z', 'И': 'Y', 'І': 'I', 'Ї': 'Yi', 'Й': 'Y', 'К': 'K', 'Л': 'L', 'М': 'M', 'Н': 'N', 'О': 'O', 'П': 'P', 'Р': 'R', 'С': 'S', 'Т': 'T', 'У': 'U', 'Ф': 'F', 'Х': 'Kh', 'Ц': 'Ts', 'Ч': 'Ch', 'Ch': 'Ch', 'Ш': 'Sh', 'Щ': 'Shch', 'Ю': 'Yu', 'Я': 'Ya', 'а': 'a', 'б': 'b', 'в': 'v', 'г': 'h', 'ґ': 'g', 'д': 'd', 'е': 'e', 'є': 'ye', 'ж': 'zh', 'з': 'z', 'и': 'y', 'і': 'i', 'ї': 'yi', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch', 'ь': '', 'ъ': '', 'ю': 'yu', 'я': 'ya', 'ё': 'yo', 'э': 'e'}
    CYR_TO_LAT_LOOKALIKES = {'А': 'A', 'В': 'B', 'Е': 'E', 'К': 'K', 'М': 'M', 'Н': 'H', 'О': 'O', 'Р': 'P', 'С': 'C', 'Т': 'T', 'Х': 'X', 'І': 'I', 'а': 'a', 'в': 'b', 'е': 'e', 'к': 'k', 'м': 'м', 'н': 'h', 'о': 'o', 'р': 'p', 'с': 'c', 'т': 't', 'х': 'x', 'і': 'і'}
    LAT_TO_CYR_LOOKALIKES = {'A': 'А', 'B': 'В', 'E': 'Е', 'K': 'К', 'M': 'М', 'H': 'Н', 'O': 'О', 'P': 'Р', 'C': 'С', 'T': 'Т', 'X': 'Х', 'I': 'І', 'a': 'а', 'b': 'в', 'e': 'е', 'k': 'к', 'm': 'м', 'h': 'н', 'o': 'о', 'p': 'p', 'c': 'с', 't': 't', 'x': 'x', 'i': 'і'}

    def __init__(self):
        self.df = None
        self.change_count = 0
        self.folder_path = os.getcwd()
        self.widgets_map = {}
        self.current_file_name = None
        self.current_sheet_name = 0
        self.ui_state = {
            'column_select': None, 
            'action_value': None, 
            'find_replace': {'mode_radio': 'substring', 'whole_cell_radio': 'part', 'regex_check': False, 'case_sensitive_check': False, 'find_input': '', 'replace_input': ''},
            'date_transform': {
                'source_parts': ['День (DD)', '- (Тире)', 'Місяць (MM)', '- (Тире)', 'Рік (YYYY)', '--- Вибрати ---'], 
                'target_parts': ['Рік (YYYY)', '- (Тире)', 'Місяць (MM)', '- (Тире)', 'День (DD)', '--- Вибрати ---'],

                'target_time_parts': ['Година (HH)', ': (Двокрапка)', 'Хвилина (MM)', ': (Двокрапка)', 'Секунда (SS)', '. (Крапка)', 'Мікросекунди (f)'],
                'date_mode': 'soft_convert',
                'keep_time': False 
            }
        }
        self.df_history = []
        self.history_index = -1
        self.history_limit = 15
        self.last_nan_indices = None
        self._create_widgets() 

    def _get_json_value(self, value_dict):
        if not isinstance(value_dict, dict):
            return None
        for key in ['string_value', 'int_value', 'float_value', 'double_value']:
            value = value_dict.get(key)

            if value is not None and str(value).lower() not in ('null', ''):
                return value
        return None

    def _flatten_ga4_event_params(self, json_string, root_key):
        if pd.isna(json_string) or not json_string:
            return {}
        
        cleaned_json_string = str(json_string).replace('\\n', '').strip()
        cleaned_json_string = cleaned_json_string.replace('""', '"')
        
        try:
            data = json.loads(cleaned_json_string)
        except json.JSONDecodeError:
            return {}

        if root_key not in data or not isinstance(data[root_key], list):
            return {}
        
        flat_data = {}
        for item in data[root_key]:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = str(item['key'])
                value = self._get_json_value(item['value'])
                if key and value is not None:
                    flat_data[key] = value
                    
        return flat_data

    def _create_widgets(self):
        self.widgets_map['folder_input'] = widgets.Text(description='Шлях до папки:', value=self.folder_path, placeholder='Введіть шлях до папки з файлами', layout=widgets.Layout(width='auto'))
        self.widgets_map['submit_folder_btn'] = widgets.Button(description='Встановити Папку', button_style='success', layout=widgets.Layout(width='auto'))
        self.widgets_map['file_dropdown'] = widgets.Dropdown(options=['--- Вибрати Файл ---'], description='Файл:', layout=widgets.Layout(width='auto'))
        self.widgets_map['sheet_dropdown'] = widgets.Dropdown(options=[], description='Аркуш:', layout=widgets.Layout(width='auto', display='none'))
        self.widgets_map['header_input'] = widgets.IntText(description='Рядок заголовків (0 - 1-й рядок, -1 - Без заголовка):', value=0, min=-1, layout=widgets.Layout(width='auto'))
        
        self.widgets_map['encoding_input'] = widgets.Text(description='Кодування CSV:', value='utf-8', placeholder='Наприклад: utf-8, windows-1251, cp1251, latin-1', layout=widgets.Layout(width='auto'))
        self.widgets_map['encoding_help'] = widgets.HTML(
            value='<small>Поширені кодування: <b>utf-8</b> (за замовчуванням), <b>windows-1251</b>, <b>cp1251</b>, <b>koi8-r</b>, <b>latin-1</b>.</small>',
            layout=widgets.Layout(margin='0 0 5px 0', display='none')
        )

        self.widgets_map['load_file_btn'] = widgets.Button(description='Завантажити Файл', button_style='info', layout=widgets.Layout(width='auto'))
        
        self.widgets_map['json_column_select'] = widgets.Dropdown(options=[], description='JSON-стовпець:', layout=widgets.Layout(width='auto'))
        self.widgets_map['json_flatten_mode'] = widgets.RadioButtons(
            options={
                'GA4 Event Params (list of key-value objects)': 'ga4_params',
                'Простий JSON-об’єкт (одна колонка)': 'simple_json'
            },
            description='Режим Розпакування:',
            value='ga4_params',
            layout=widgets.Layout(width='auto')
        )
        self.widgets_map['json_target_column_input'] = widgets.Text(description='Кореневий ключ (для GA4):', value='event_params', placeholder='Наприклад: event_params', layout=widgets.Layout(width='auto'))
        self.widgets_map['json_apply_btn'] = widgets.Button(description='Розпакувати JSON', button_style='primary', layout=widgets.Layout(width='auto'))

        self.widgets_map['action_dropdown'] = widgets.Dropdown(
            options={
                '1. Очистити Дублікати': 'drop_duplicates',
                '2. Замінити NaN на 0 (Числа)': 'fillna_zero',
                '3. Транслітерація (Кирилиця <> Латиниця)': 'transliterate',
                '4. Нормалізувати Пробіли (прибрати зайві)': 'normalize_spaces',
                '5. Нормалізувати Регістр (Title/Upper/Lower/Sentence)': 'normalize_case',
                '6. Модифікація Дат/Часу': 'to_datetime',
                '7. Керування Стовпцями та Даними (Перейменування/Видалення)': 'data_management',
                '8. Заповнити Пропуски (Медіана/Середнє/Мода)': 'fill_missing_stats',
                '9. Аналіз Унікальних Значень та Розподілу (Графік)': 'check_unique',
                '10. Пошук та Заміна Тексту': 'find_replace',
                '11. Фільтрування (Видалення Не Відповідних)': 'filter_data',
                '12. Аналіз Рядків з NaN (Ручне Редагування)': 'analyze_nan',
                '13. Сортування Даних': 'sort_data',
                '14. Зберегти Файл': 'save_file',
                '15. Розпакувати JSON-стовпець (GA4-формат)': 'flatten_json_column' 
            },
            description='Виберіть Дію:',
            value='drop_duplicates',
            layout=widgets.Layout(width='auto')
        )
        self.widgets_map['dynamic_controls'] = widgets.Output() 
        self.widgets_map['undo_btn'] = widgets.Button(description='⏪ Відмінити (Undo)', button_style='warning', layout=widgets.Layout(width='150px'))
        self.widgets_map['redo_btn'] = widgets.Button(description='Повторити (Redo) ⏩', button_style='warning', layout=widgets.Layout(width='150px'))
        
        self.widgets_map['nan_column_select'] = widgets.Dropdown(options=[], description='Стовпець для аналізу:', layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_limit_input'] = widgets.IntText(description='Ліміт рядків:', value=10, min=1, layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_indices_remove'] = widgets.Text(description='Індекси для видалення (0, 3-5):', placeholder='Введіть індекси відображення', layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_remove_btn'] = widgets.Button(description='Видалити Рядки', button_style='danger', layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_column_select_edit'] = widgets.Dropdown(options=[], description='Стовпець для заміни:', layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_indices_replace'] = widgets.Text(description='Індекси для заміни (0, 3-5):', placeholder='Введіть індекси відображення', layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_replace_value'] = widgets.Text(description='Нове значення:', placeholder='Наприклад, 0 або NONE', layout=widgets.Layout(width='auto'))
        self.widgets_map['nan_replace_btn'] = widgets.Button(description='Замінити Значення', button_style='info', layout=widgets.Layout(width='auto'))
        
        self.widgets_map['save_filename_input'] = widgets.Text(description='Ім\'я файлу:', placeholder='cleaned_data.xlsx', layout=widgets.Layout(width='auto'))
        self.widgets_map['save_btn'] = widgets.Button(description='Зберегти Файл', button_style='success', layout=widgets.Layout(width='auto'))

        self.widgets_map['main_output'] = widgets.Output()
        self.widgets_map['file_output'] = widgets.VBox(layout=widgets.Layout(border='1px solid lightgray', padding='10px', margin='10px 0'))
        self.widgets_map['load_settings_container'] = widgets.VBox(layout=widgets.Layout(display='none'))
        self.widgets_map['action_panel'] = widgets.VBox([
            widgets.HBox([self.widgets_map['action_dropdown'], self.widgets_map['undo_btn'], self.widgets_map['redo_btn']]),
            self.widgets_map['dynamic_controls']
        ], layout=widgets.Layout(border='1px solid lightgray', padding='10px', margin='10px 0', display='none'))
        self.widgets_map['results_output'] = widgets.Output(layout=widgets.Layout(border='1px solid lightcoral', padding='10px', margin='10px 0', min_height='100px'))
        self.widgets_map['df_display_output'] = widgets.Output()
        self.widgets_map['summary_accordion'] = widgets.Accordion(children=[self.widgets_map['df_display_output']], layout=widgets.Layout(display='none'))
        self.widgets_map['summary_accordion'].set_title(0, "📋 Поточний Стан Даних")

        self.widgets_map['submit_folder_btn'].on_click(self._handle_folder_submit)
        self.widgets_map['file_dropdown'].observe(self._handle_file_select, names='value')
        self.widgets_map['load_file_btn'].on_click(self._handle_file_load)
        self.widgets_map['action_dropdown'].observe(self._update_action_ui, names='value')
        self.widgets_map['undo_btn'].on_click(self._handle_undo)
        self.widgets_map['redo_btn'].on_click(self._handle_redo)
        self.widgets_map['nan_remove_btn'].on_click(lambda b: self._handle_action_apply('remove_nan_rows', column=self.widgets_map['nan_column_select'].value, index_str=self.widgets_map['nan_indices_remove'].value))
        self.widgets_map['nan_replace_btn'].on_click(lambda b: self._handle_action_apply('replace_nan_values', column=self.widgets_map['nan_column_select'].value, edit_column=self.widgets_map['nan_column_select_edit'].value, index_str=self.widgets_map['nan_indices_replace'].value, new_value=self.widgets_map['nan_replace_value'].value))
        self.widgets_map['save_btn'].on_click(self._handle_save_file)
        self.widgets_map['json_apply_btn'].on_click(self._handle_json_flatten_apply)

    def _handle_json_flatten_apply(self, b):
        self._handle_action_apply(
            'flatten_json_column',
            column=self.widgets_map['json_column_select'].value,
            mode=self.widgets_map['json_flatten_mode'].value,
            root_key=self.widgets_map['json_target_column_input'].value
        )

    def log_change(self, message):
        self.change_count += 1
        logger.info(f"[Зміна {self.change_count}] {message}")

    def _save_state(self):
        if self.df is None: return
        if self.history_index < len(self.df_history) - 1:
            self.df_history = self.df_history[:self.history_index + 1]
        self.df_history.append(self.df.copy())
        if len(self.df_history) > self.history_limit: self.df_history.pop(0)
        self.history_index = len(self.df_history) - 1
        self.change_count = self.history_index

    def _handle_undo(self, b):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.history_index > 0:
                self.history_index -= 1
                self.df = self.df_history[self.history_index].copy()
                logger.info(f"🔄 Відміна дії. Повернуто до стану №{self.history_index}.")
                self._display_summary()
                self._update_action_ui(None)
            else:
                logger.warning("⚠️ Немає дій для відміни.")

    def _handle_redo(self, b):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.history_index < len(self.df_history) - 1:
                self.history_index += 1
                self.df = self.df_history[self.history_index].copy()
                logger.info(f"➡️ Повтор дії. Перехід до стану №{self.history_index}.")
                self._display_summary()
                self._update_action_ui(None)
            else:
                logger.warning("⚠️ Немає дій для повтору.")

    def _get_column_options(self, include_all=False, dtype_include=None):
        if self.df is None: return []
        columns = list(self.df.columns)
        
        filtered_columns = list(columns)
        all_option_label = None

        if dtype_include == 'numeric':
            filtered_columns = [col for col in filtered_columns if pd.api.types.is_numeric_dtype(self.df[col])]
            all_option_label = '--- Всі числові стовпці ---'
        elif dtype_include == 'text':
            filtered_columns = [col for col in filtered_columns if self.df[col].dtype in ['object', 'string'] or isinstance(self.df[col].dtype, pd.CategoricalDtype)]
            all_option_label = '--- Всі текстові стовпці ---'
        
        if include_all and all_option_label and filtered_columns:
             filtered_columns.insert(0, all_option_label)
             
        return filtered_columns

    def _get_all_files(self):
        try:
            files = [f for f in os.listdir(self.folder_path) if os.path.isfile(os.path.join(self.folder_path, f))]
            return [f for f in files if os.path.splitext(f)[1].lower() in ['.csv', '.xls', '.xlsx'] and not f.startswith('~')]
        except (FileNotFoundError, Exception):
            return []

    def _display_summary(self):
        with self.widgets_map['df_display_output']:
            clear_output(wait=True)
            if self.df is None:
                display(HTML("<h2>🚫 Дані не завантажено.</h2>"))
                return
            
            self.widgets_map['summary_accordion'].set_title(0, f"📋 Поточний Стан Даних (Стан №{self.history_index} | {len(self.df)} рядків, {len(self.df.columns)} стовпців)")
                        
            buf = io.StringIO()
            self.df.info(buf=buf, verbose=False, memory_usage=False, show_counts=True)
            info_str = buf.getvalue().split('\n')
            
            info_html = "<h4>Схема даних:</h4>"
            info_html += "<table>"
            for line in info_str[3:-2]:
                parts = line.split()
                if len(parts) >= 3:
                    col_index = parts[0]
                    col_name = parts[1]
                    non_null_count = parts[2]
                    dtype = parts[-1]
                    info_html += f"<tr><td>{col_index}</td><td>**{col_name}**</td><td>{non_null_count} Non-Null</td><td>{dtype}</td></tr>"
            info_html += "</table>"
            display(HTML(info_html))
            
            display(HTML("<h4>Початок датасету (Перші 5 рядків):</h4>"))
            display(HTML(self.df.head().to_html()))

            display(HTML("<h4>Кінець датасету (Останні 5 рядків):</h4>"))
            display(HTML(self.df.tail().to_html()))

            display(HTML("<h4>Описова статистика:</h4>"))
            try:
                display(HTML(self.df.describe(include='all').to_html()))
            except:
                display(HTML("<p>Неможливо відобразити описову статистику.</p>"))

    # ==============================================================================
    
    def _aggressively_clean_datetime(self, text, user_separators):
        if pd.isna(text): return None
        text_str = str(text)
        
        allowed_separators = set(user_separators)
        allowed_separators.update(['.', ':', '-', '/', ' '])
        
        allowed_chars_pattern = r'[^0-9\\s' + re.escape("".join(allowed_separators)) + r']'
        
        cleaned_text = re.sub(allowed_chars_pattern, '', text_str, flags=re.IGNORECASE)
        
        cleaned_text = re.sub(r'\\s+', ' ', cleaned_text).strip()
        
        return cleaned_text if cleaned_text else None
        
    def _extract_and_normalize_by_mask(self, text, source_parts_raw):
        """
        Режим 3: Використовує Regex на основі вхідного шаблону для вилучення чистих частин
        дати/часу.
        """
        if pd.isna(text): return None
        text_str = str(text)
        regex_pattern = ""
        group_identifiers = []
        part_to_regex_map = {
            '%Y': r'(\d{4})',  
            '%y': r'(\d{2})',  
            '%m': r'(\d{1,2})', 
            '%d': r'(\d{1,2})', 
            '%H': r'(\d{1,2})', 
            '%M': r'(\d{1,2})', 
            '%S': r'(\d{1,2})', 
            '%f': r'(\d{1,6})(?:[^\d\s]*)?' 
        }
        
        for part in source_parts_raw:
            if part == '--- Вибрати ---': continue
            
            if part in self.DATE_ELEMENTS and self.DATE_ELEMENTS[part]:
                format_code = self.DATE_ELEMENTS[part]
                
                regex_pattern += part_to_regex_map.get(format_code, r'(\d+)') 
                group_identifiers.append(format_code)
                
            elif part in self.DATE_SEPARATORS and self.DATE_SEPARATORS[part]:
                separator = self.DATE_SEPARATORS[part]
                
                if separator == ' ':
                     regex_pattern += r'\s+'
                else:
                     escaped_sep = re.escape(separator)
                     regex_pattern += r'[^\d\s]*?' + escaped_sep + r'[^\d\s]*?'

        final_regex = r'.*?' + regex_pattern + r'.*?' 
        match = re.search(final_regex, text_str, re.IGNORECASE)
        
        if not match:
            return None

        groups = list(match.groups()) 
        reconstructed_parts = []
        group_index = 0
        
        for part in source_parts_raw:
            if part == '--- Вибрати ---': continue
            
            if part in self.DATE_ELEMENTS and self.DATE_ELEMENTS[part]:
                format_code = self.DATE_ELEMENTS[part]
                
                value = groups[group_index] if group_index < len(groups) else None
                
                if format_code in ['%H', '%M', '%S', '%f'] and value is not None:
                     value = re.sub(r'[^\d]', '', str(value))
                     
                if value is None or not str(value).strip():
                    if format_code in ['%H', '%M', '%S']: 
                        value = '00' 
                    elif format_code == '%f':
                        value = '000000' 
                    else:
                        return None 

                if format_code in ['%m', '%d', '%H', '%M', '%S'] and len(str(value)) <= 2:
                    reconstructed_parts.append(str(value).zfill(2))
                
                elif format_code == '%f':
                     value_str = str(value)
                     reconstructed_parts.append(value_str.ljust(6, '0')[:6])
                
                else:
                    reconstructed_parts.append(str(value))
                
                group_index += 1
            
            elif part in self.DATE_SEPARATORS and self.DATE_SEPARATORS[part]:
                reconstructed_parts.append(self.DATE_SEPARATORS[part])
                
        return "".join(reconstructed_parts)
    
    
    def drop_duplicates_core(self, keep):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            rows_to_remove = self.df.duplicated(keep=keep).sum()
            if rows_to_remove == 0:
                logger.warning("⚠️ Дублікатів не знайдено.")
                return

            logger.info(f"🔎 Знайдено **{rows_to_remove}** дублікатів. Виконую видалення...")
            
            rows_before = len(self.df)
            self._save_state()
            self.df.drop_duplicates(keep=keep, inplace=True)
            rows_removed = rows_before - len(self.df)
            
            if rows_removed > 0:
                self.log_change(f"Видалено {rows_removed} дублікатів. Залишено: {keep}.")
                logger.info(f"✅ Видалено **{rows_removed}** рядків-дублікатів. Залишено: **{keep}**.")
            else:
                logger.error("❌ Помилка видалення: дублікати були знайдені, але не видалені.")
                self.df_history.pop()
                self.history_index -= 1
            

    def fillna_zero_core(self, column):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            if column == '--- Всі числові стовпці ---':
                cols_to_process = self._get_column_options(dtype_include='numeric')
                if not cols_to_process:
                    logger.warning("⚠️ Числових стовпців для обробки не знайдено.")
                    return
            else:
                if not pd.api.types.is_numeric_dtype(self.df[column]):
                    logger.error(f"❌ Стовпець **{column}** не є числовим.")
                    return
                cols_to_process = [column]

            total_nan_filled = 0
            self._save_state()
            for col in cols_to_process:
                initial_nan_count = self.df[col].isnull().sum()
                if initial_nan_count > 0:
                    self.df[col].fillna(0, inplace=True)
                    total_nan_filled += initial_nan_count
            
            if total_nan_filled > 0:
                self.log_change(f"Замінено {total_nan_filled} NaN на 0 у {len(cols_to_process)} стовпцях.")
                logger.info(f"✅ Замінено **{total_nan_filled}** NaN на **0** у {len(cols_to_process)} стовпцях.")
            else:
                self.df_history.pop()
                self.history_index -= 1

    def transliterate_core(self, column, mode):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.df[column].dtype.kind not in 'OSU':
                logger.error(f"❌ Стовпець {column} не є текстовим.")
                return

            if mode == 'full':
                mapping = self.CYR_TO_LAT_FULL
                mode_name = 'Повна (ру/укр на лат)'
            elif mode == 'cyr_to_lat_lookalikes': 
                mapping = self.CYR_TO_LAT_LOOKALIKES
                mode_name = 'Схожі ру/укр на лат'
            elif mode == 'lat_to_cyr_lookalikes': 
                mapping = self.LAT_TO_CYR_LOOKALIKES
                mode_name = 'Схожі лат на ру/укр'
            else:
                 logger.error("❌ Невідомий режим транслітерації.")
                 return
            
            def replace_chars(text):
                if pd.isna(text): return text
                return ''.join([mapping.get(c, c) for c in str(text)])

            original_series = self.df[column].astype(str).copy()
            self._save_state()
            self.df[column] = self.df[column].apply(replace_chars)
            
            changed_count = (original_series != self.df[column].astype(str)).sum()
            if changed_count > 0:
                self.log_change(f"Транслітерація ({mode_name}) у {column}. Змінено комірок: {changed_count}")
                logger.info(f"✅ У стовпці **{column}** виконано транслітерацію ({mode_name}). Змінено комірок: **{changed_count}**")
            else:
                logger.warning(f"⚠️ Транслітерація не змінила жодної комірки у стовпці {column}.")
                self.df_history.pop()
                self.history_index -= 1

    def normalize_spaces_core(self, column):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            if column == '--- Всі текстові стовпці ---':
                cols_to_process = self._get_column_options(dtype_include='text')
                target_cols_log = 'у всіх текстових стовпцях'
                if not cols_to_process:
                    logger.warning("⚠️ Текстових стовпців для обробки не знайдено.")
                    return
            else:
                if self.df[column].dtype.kind not in 'OSU':
                    logger.error(f"❌ Стовпець {column} не є текстовим.")
                    return
                cols_to_process = [column]
                target_cols_log = f'у стовпці **{column}**'

            total_changed_count = 0
            self._save_state()
            for col in cols_to_process:
                original_series = self.df[col].astype(str).copy()
                self.df[col] = self.df[col].astype(str).str.replace(r'\\s+', ' ', regex=True).str.strip()
                changed_count = (original_series != self.df[col].astype(str)).sum()
                total_changed_count += changed_count
            
            if total_changed_count > 0:
                self.log_change(f"Нормалізовано пробіли {target_cols_log}. Змінено комірок: {total_changed_count}")
                logger.info(f"✅ Пробіли нормалізовано {target_cols_log}. Змінено комірок: **{total_changed_count}**")
            else:
                logger.warning(f"⚠️ Зайвих пробілів {target_cols_log} не знайдено.")
                self.df_history.pop()
                self.history_index -= 1
            
    def normalize_case_core(self, column, case_type):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.df[column].dtype.kind not in 'OSU':
                logger.error(f"❌ Стовпець {column} не є текстовим.")
                return
            case_map = {'Title Case (Кожне слово з великої)': 'title', 'UPPER CASE (ВСІ ВЕЛИКІ)': 'upper', 'lower case (всі малі)': 'lower', 'Sentence Case (Перший символ великий)': 'capitalize'}
            case_func_name = case_map.get(case_type)
            
            original_series = self.df[column].astype(str).copy()
            self._save_state()
            self.df[column] = self.df[column].astype(str).str.__getattribute__(case_func_name)()
            
            changed_count = (original_series != self.df[column].astype(str)).sum()
            if changed_count > 0:
                self.log_change(f"Нормалізовано регістр у {column}. Змінено комірок: {changed_count}")
                logger.info(f"✅ Регістр нормалізовано у стовпці **{column}**. Змінено комірок: **{changed_count}**")
            else:
                logger.warning(f"⚠️ Змін регістру у стовпці {column} не виявлено.")
                self.df_history.pop()
                self.history_index -= 1
                
    def to_datetime_core(self, column, source_parts, target_parts, output_mode, keep_time, date_mode):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            source_parts_raw = source_parts 
            source_format_parts = []
            source_separators_codes = []
            
            for i in range(len(source_parts_raw)):
                 part = source_parts_raw[i]
                 if part == '--- Вибрати ---': continue
                 
                 format_code = self.DATE_ELEMENTS.get(part) or self.DATE_SEPARATORS.get(part)
                 source_format_parts.append(format_code)
                 
                 if self.DATE_SEPARATORS.get(part):
                      source_separators_codes.append(self.DATE_SEPARATORS.get(part))
                      
            source_format = "".join(source_format_parts)

            if not source_format and date_mode == 'hard_convert_pattern':
                logger.error("❌ Необхідно вказати ВХІДНИЙ формат даних для Режиму 3 (Жорстка).")
                return
            
            original_series = self.df[column].astype(str).copy() 

            try:
                if date_mode == 'hard_convert_pattern':
                    logger.info("⚙️ Використовується режим **3. Жорстка (Вилучення та Нормалізація за Патерном)**.")
                    data_to_convert = self.df[column].apply(lambda x: self._extract_and_normalize_by_mask(x, source_parts_raw))
                
                elif date_mode == 'soft_clean_infer':
                    logger.info("⚙️ Використовується режим **2. М'яке Очищення та Авто-визначення Формату (Inference)**.")
                    data_to_convert = self.df[column].apply(lambda x: self._aggressively_clean_datetime(x, source_separators_codes))
                    source_format = None
                    
                else: 
                    logger.info("⚙️ Використовується режим **1. Стандартна (Pandas Auto, для чистих/змішаних дат)**.")
                    data_to_convert = self.df[column]
                    source_format = None
                
                if source_format is None:
                     temp_series = pd.to_datetime(data_to_convert, errors='coerce') 
                else:
                     temp_series = pd.to_datetime(data_to_convert, format=source_format, errors='coerce')
                
                initial_nan_count = temp_series.isnull().sum()
                if initial_nan_count == len(self.df):
                    logger.error(f"❌ Не вдалося розпізнати жодного значення у стовпці {column} за форматом **'{source_format if source_format else 'AUTO'}'**.")
                    logger.error("ℹ️ Перевірте коректність вхідного формату.")
                    return
                
                self._save_state()
                mode_log_map = {'soft_convert': 'Стандартна (Авто)', 'soft_clean_infer': 'М\'яке Очищення (Авто)', 'hard_convert_pattern': 'Жорстка (Патерн)'}
                mode_log = mode_log_map.get(date_mode, 'Конвертація')

                if output_mode == 'replace_format':
                    
                    target_parts_codes = [self.DATE_ELEMENTS.get(p) or self.DATE_SEPARATORS.get(p) for p in target_parts if p != '--- Вибрати ---']
                    target_format = ''.join(target_parts_codes)
                    
                    if not target_format:
                        logger.error("❌ Необхідно вказати ВИХІДНИЙ формат для конвертації.")
                        self.df_history.pop()
                        self.history_index -= 1
                        return
                    
                    log_time_status = "(з часом)" if keep_time else "(без часу)"

                    if not keep_time:
                         temp_series = temp_series.dt.normalize()
                    
                    formatted_series = temp_series.dt.strftime(target_format)
                    
                    self.df[column] = np.where(temp_series.notna(), formatted_series, original_series)
                    
                    changed_count = (original_series != self.df[column].astype(str)).sum()
                    
                    self.log_change(f"Конвертовано ({mode_log}) та змінено формат стовпця {column} на {target_format} {log_time_status}. Змінено: {changed_count}. (Не розпізнано: {initial_nan_count})")
                    logger.info(f"✅ Стовпець **{column}** конвертовано ({mode_log}) та змінено формат на **'{target_format}'** {log_time_status}. (Змінено комірок: **{changed_count}**, Не розпізнано значень: **{initial_nan_count}**)")

                
                elif output_mode == 'create_new':
                    
                    self.df[column] = np.where(temp_series.isna(), original_series, self.df[column])
                    
                    target_map = {'Рік': 'year', 'Місяць': 'month', 'День': 'day', 'Година': 'hour', 'Хвилина': 'minute', 'Секунда': 'second'}
                    new_cols_created = 0
                    for part_name in target_parts:
                        if part_name not in target_map: continue
                        part_attr = target_map[part_name]
                        new_col_name = f"{column}_{part_name.lower()}"
                        if new_col_name in self.df.columns:
                            logger.warning(f"⚠️ Стовпець '{new_col_name}' вже існує. Пропускаю.")
                            continue
                        self.df[new_col_name] = temp_series.dt.__getattribute__(part_attr)
                        new_cols_created += 1

                    if new_cols_created > 0:
                        self.log_change(f"Створено {new_cols_created} нових стовпців з частин дати/часу {column}. (Не розпізнано: {initial_nan_count})")
                        logger.info(f"✅ Створено **{new_cols_created}** нових стовпців з частин дати/часу. (Не розпізнано значень: **{initial_nan_count}**)")
                    else:
                        logger.warning("⚠️ Не створено нових стовпців.")
                        self.df_history.pop()
                        self.history_index -= 1

            except ValueError as ve:
                logger.error(f"❌ Помилка формату дати: {ve}. Перевірте відповідність формату даним.")
                if self.history_index >= 0:
                    self.df_history.pop()
                    self.history_index -= 1
            except Exception as e:
                logger.error(f"❌ Критична помилка конвертації дати/часу: {e}.")
                if self.history_index >= 0:
                    self.df_history.pop()
                    self.history_index -= 1
            
    def drop_data_core(self, mode, column=None):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            rows_before = len(self.df)
            
            if mode == 'column' and column:
                self._save_state()
                self.df.drop(columns=[column], inplace=True)
                self.log_change(f"Видалено стовпець: {column}.")
                logger.info(f"✅ Видалено стовпець **{column}**.")
            elif mode == 'rows_with_nan':
                self._save_state()
                self.df.dropna(how='any', inplace=True)
                rows_removed = rows_before - len(self.df)
                if rows_removed > 0:
                    self.log_change(f"Видалено {rows_removed} рядків, що містили NaN.")
                    logger.info(f"✅ Видалено **{rows_removed}** рядків з NaN.")
                else:
                    logger.warning("⚠️ Не знайдено рядків з NaN для видалення.")
                    self.df_history.pop()
                    self.history_index -= 1
            else:
                 logger.error("❌ Невідомий режим видалення.")
                 
    def rename_column_core(self, old_name, new_name):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if not old_name or not new_name.strip():
                logger.error("❌ Імена стовпців не можуть бути порожніми.")
                return
            if new_name in self.df.columns and old_name != new_name:
                logger.error(f"❌ Стовпець з ім'ям '{new_name}' вже існує.")
                return
            self._save_state()
            self.df.rename(columns={old_name: new_name}, inplace=True)
            self.log_change(f"Перейменовано стовпець '{old_name}' на '{new_name}'.")
            logger.info(f"✅ Стовпець '{old_name}' успішно перейменовано на '{new_name}'.")
            
    def find_replace_column_names_core(self, find_value, replace_value):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            find_str = str(find_value)
            replace_str = str(replace_value)
            
            new_columns = []
            changed_count = 0
            
            for col in self.df.columns:
                new_col = col.replace(find_str, replace_str)
                new_columns.append(new_col)
                if new_col != col:
                    changed_count += 1
            
            if changed_count == 0:
                logger.warning(f"⚠️ Підрядок '{find_str}' не знайдено в жодній назві стовпця.")
                return

            if len(set(new_columns)) != len(new_columns):
                logger.error("❌ Заміна призводить до дублікатів назв стовпців. Операцію скасовано.")
                return

            self._save_state()
            self.df.columns = new_columns
            self.log_change(f"У назвах стовпців замінено '{find_str}' на '{replace_str}'. Змінено: {changed_count}.")
            logger.info(f"✅ У назвах стовпців замінено підрядок. Змінено **{changed_count}** назв.")

    def normalize_column_names_case_core(self, case_type):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            new_columns = []
            changed_count = 0
            
            self._save_state()
            
            for col in self.df.columns:
                original_col = col
                if case_type == 'title':
                    new_col = col.title()
                elif case_type == 'lower':
                    new_col = col.lower()
                elif case_type == 'sentence': 
                    if len(col) > 0:
                        new_col = col[0].upper() + col[1:].lower()
                    else:
                        new_col = col
                else:
                    logger.error("❌ Невідомий тип регістру.")
                    self.df_history.pop()
                    self.history_index -= 1
                    return

                new_columns.append(new_col)
                if new_col != original_col:
                    changed_count += 1
            
            if changed_count == 0:
                logger.warning(f"⚠️ Назви стовпців вже відповідають регістру '{case_type}'.")
                self.df_history.pop()
                self.history_index -= 1
                return

            if len(set(new_columns)) != len(new_columns):
                logger.error("❌ Зміна регістру призводить до дублікатів назв стовпців. Операцію скасовано.")
                self.df_history.pop()
                self.history_index -= 1
                return

            self.df.columns = new_columns
            self.log_change(f"Змінено регістр назв стовпців на {case_type}. Змінено: {changed_count}.")
            logger.info(f"✅ Змінено регістр назв стовпців на **{case_type}**. Змінено **{changed_count}** назв.")
        
    def fill_missing_stats_core(self, column, method):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            initial_nan_count = self.df[column].isnull().sum()
            if initial_nan_count == 0:
                logger.warning(f"У стовпці {column} пропущених значень не знайдено.")
                return

            fill_value = None
            log_message = ""
            is_numeric = pd.api.types.is_numeric_dtype(self.df[column])

            try:
                if method == 'median' and is_numeric:
                    fill_value = self.df[column].median()
                    log_message = f"медіаною ({fill_value})"
                elif method == 'mean' and is_numeric:
                    fill_value = self.df[column].mean()
                    log_message = f"середнім ({fill_value:.2f})"
                elif method == 'min' and is_numeric:
                    fill_value = self.df[column].min()
                    log_message = f"мінімумом ({fill_value})"
                elif method == 'max' and is_numeric:
                    fill_value = self.df[column].max()
                    log_message = f"максимумом ({fill_value})"
                elif method == 'mode':
                    mode_values = self.df[column].mode()
                    if not mode_values.empty:
                        fill_value = mode_values[0] 
                        log_message = f"модою ('{fill_value}')"
                    else:
                        logger.error("❌ Не вдалося знайти моду для заповнення.")
                        return
                else:
                    logger.error(f"❌ Метод '{method}' не застосовний до типу даних стовпця {column}.")
                    return
                
                if pd.isna(fill_value) and method in ['median', 'mean', 'min', 'max']:
                    logger.error(f"❌ Отримане значення для заповнення ({method}) є NaN. Неможливо виконати операцію.")
                    return

                self._save_state()
                self.df[column].fillna(fill_value, inplace=True)
                
                self.log_change(f"Заповнено {initial_nan_count} NaN у {column} за допомогою {log_message}.")
                logger.info(f"✅ Заповнено **{initial_nan_count}** NaN у **{column}** за допомогою **{log_message}**.")
            
            except Exception as e:
                logger.error(f"❌ Помилка при заповненні пропусків: {e}")
                self.df_history.pop()
                self.history_index -= 1
            
    
    def generate_unique_values(self, column):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            
            if self.df[column].isnull().all():
                logger.warning(f"⚠️ Стовпець {column} містить лише пропущені значення (NaN).")
                self._display_summary()
                return
            
            if pd.api.types.is_numeric_dtype(self.df[column]):
                
                stats = self.df[column].describe().to_frame().T
                logger.info(f"📊 Числова статистика для стовпця **{column}**:")
                display(HTML(stats.to_html()))
                
                logger.info("📈 Візуалізація розподілу (Гістограма):")
                plt.figure(figsize=(8, 4))
                
                data = self.df[column].dropna()
                
                if len(data.unique()) < 20 and len(data.unique()) > 1:
                    bins = len(data.unique())
                else:
                    bins = 'auto'
                    
                plt.hist(data, bins=bins, edgecolor='black')
                plt.title(f'Розподіл значень у стовпці: {column}')
                plt.xlabel(column)
                plt.ylabel('Кількість')
                plt.grid(axis='y', alpha=0.75)
                plt.show()

            else:
                unique_counts = self.df[column].value_counts(dropna=False)
                unique_counts.index = unique_counts.index.fillna('**[NaN/Пропущене]**')
                top_50 = unique_counts.nlargest(50).to_frame(name='К-сть')
                total_unique = len(unique_counts)
                
                logger.info(f"📋 Унікальні значення для стовпця **{column}** (всього {total_unique}):")
                display(HTML(top_50.to_html()))
                
                if total_unique > 50: logger.warning(f"⚠️ Відображено лише перші 50 з {total_unique} унікальних значень.")
            
            self._display_summary()

    def find_and_replace_core(self, column, find_value, replace_value, mode, use_regex, whole_cell_match, case_sensitive):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.df[column].dtype.kind not in 'OSU': 
                logger.error(f"❌ Стовпець {column} не є текстовим.")
                return

            self.ui_state['find_replace'].update({'find_input': str(find_value), 'replace_input': str(replace_value), 'mode_radio': mode, 'regex_check': use_regex, 'whole_cell_radio': 'whole' if whole_cell_match else 'part', 'case_sensitive_check': case_sensitive})
            
            changed_count = 0
            if mode == 'unique':
                mask = self.df[column].isnull() if pd.isna(find_value) or str(find_value).lower() in ['nan', 'nan/пропущене'] else (self.df[column].astype(str) == str(find_value))
                
                original_series = self.df[column].astype(str).copy()
                
                self._save_state()
                self.df.loc[mask, column] = replace_value
                
                changed_count = (original_series != self.df[column].astype(str)).sum()
                
                if changed_count > 0:
                    self.log_change(f"Замінено {changed_count} унікальних значень '{find_value}' на '{replace_value}' у {column}.")
                    logger.info(f"✅ У стовпці **{column}** замінено **{changed_count}** унікальних значень.")
                else:
                    self.df_history.pop()
                    self.history_index -= 1
                    logger.warning(f"⚠️ У стовпці {column} значення '{find_value}' не знайдено.")

            else: 
                pattern = str(find_value) if use_regex else re.escape(str(find_value))
                if whole_cell_match: pattern = f'^{pattern}$'
                
                original_series = self.df[column].astype(str).copy()
                new_col = self.df[column].astype(str).str.replace(pattern, str(replace_value), case=case_sensitive, regex=True)
                
                self._save_state()
                self.df[column] = new_col
                
                changed_count = (original_series != self.df[column].astype(str)).sum()
                
                if changed_count > 0:
                    self.log_change(f"Замінено значення у {changed_count} комірках {column}.")
                    logger.info(f"✅ У стовпці **{column}** замінено значення у **{changed_count}** комірках.")
                else:
                    self.df_history.pop()
                    self.history_index -= 1
                    logger.warning(f"⚠️ У стовпці {column} значення '{find_value}' не знайдено.")

    def filter_data_core(self, column, condition_type, value):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if not condition_type or condition_type == '--- Виберіть Умову ---':
                logger.error("❌ Необхідно вибрати умову для фільтрації.")
                return
            try:
                df_col = self.df[column]
                mask = None
                
                if condition_type == 'is_nan': mask = df_col.isnull()
                elif condition_type == 'is_not_nan': mask = df_col.notnull()
                
                elif pd.api.types.is_numeric_dtype(df_col) and condition_type in ['equal', 'not_equal', 'greater_than', 'less_than']:
                    numeric_value = pd.to_numeric(value, errors='coerce')
                    if pd.isna(numeric_value):
                        logger.error(f"❌ '{value}' не є числовим значенням.")
                        return
                    conditions = {'equal': df_col == numeric_value, 'not_equal': df_col != numeric_value, 'greater_than': df_col > numeric_value, 'less_than': df_col < numeric_value}
                    mask = conditions.get(condition_type)
                elif condition_type in ['equal', 'not_equal', 'contains', 'not_contains']:
                    str_value = str(value)
                    conditions = {'equal': df_col.astype(str) == str_value, 'not_equal': df_col.astype(str) != str_value, 'contains': df_col.astype(str).str.contains(str_value, case=False, na=False), 'not_contains': ~df_col.astype(str).str.contains(str_value, case=False, na=False)}
                    mask = conditions.get(condition_type)
                
                if mask is None:
                    logger.error("❌ Невідомий тип умови або невідповідність типу даних стовпця.")
                    return

                if not mask.any():
                    logger.error("❌ Жоден рядок не відповідає фільтру.")
                    return

                rows_before = len(self.df)
                self._save_state()
                self.df = self.df[mask].copy()
                rows_removed = rows_before - len(self.df)
                if rows_removed > 0:
                    self.log_change(f"Фільтрація: Видалено {rows_removed} рядків.")
                    logger.info(f"✅ Фільтрація завершена. Видалено **{rows_removed}** рядків.")
                else:
                    logger.warning("⚠️ Фільтрація не видалила жодного рядка.")
                    self.df_history.pop()
                    self.history_index -= 1
            except Exception as e:
                logger.error(f"❌ Критична помилка при фільтруванні: {e}")

    def analyze_nan_core(self, column, limit, re_display=False):
        with self.widgets_map['results_output']:
            if not re_display: clear_output(wait=True)
            if column is None:
                logger.error("❌ Необхідно вибрати стовпець для аналізу NaN.")
                self._display_summary()
                return
            nan_mask = self.df[column].isnull()
            nan_count = nan_mask.sum()
            if column in self.df.columns:
                 self.widgets_map['nan_column_select_edit'].options = self._get_column_options() 
                 self.widgets_map['nan_column_select_edit'].value = column
            if nan_count == 0:
                logger.warning(f"⚠️ Стовпець **{column}** не містить пропущених значень (NaN).")
                self.last_nan_indices = None
            else:
                nan_df = self.df[nan_mask].copy()
                self.last_nan_indices = list(nan_df.index)
                logger.info(f"📋 Знайдено **{nan_count}** рядків, де **{column}** має NaN/Пропущене значення. (Показано перші {limit} рядків)")
                nan_summary = nan_df.isnull().sum()
                nan_summary = nan_summary[nan_summary > 0]
                if not nan_summary.empty:
                    nan_summary_df = nan_summary.to_frame(name='К-сть NaN').sort_values(by='К-сть NaN', ascending=False)
                    logger.info("ℹ️ Розподіл NaN в інших стовпцях для цих же рядків:")
                    display(HTML(nan_summary_df.to_html()))
                display_df = nan_df.head(limit).copy()
                display_df.index.name = 'Оригінальний Індекс DF'
                display_df.reset_index(inplace=True)
                display_df.index.name = 'Індекс Відображення'
                logger.info("⚠️ Індекси рядків (ліворуч) використовуються для Видалення/Заміни.")
                display(HTML(display_df.to_html()))
            self._display_summary()

    def _parse_index_range(self, index_str):
        if self.last_nan_indices is None: return set(), "Немає даних для аналізу NaN. Спочатку виконайте 'Показати Рядки з NaN'."
        indices_to_process, error_message = set(), None
        display_to_original_map = {i: idx for i, idx in enumerate(self.last_nan_indices)}
        max_display_index = len(self.last_nan_indices) - 1
        for part in re.split(r'[,\\s]+', index_str.strip()):
            if not part: continue
            if '-' in part:
                try:
                    start, end = map(int, part.split('-'))
                    if start > end: error_message = f"Некоректний діапазон '{part}'."
                    for i in range(start, end + 1):
                        if i in display_to_original_map: indices_to_process.add(display_to_original_map[i])
                        else: error_message = f"Некоректний індекс відображення: {i} (макс. {max_display_index})."
                except ValueError: error_message = f"Некоректний формат діапазону '{part}'."
            else:
                try:
                    index = int(part)
                    if index in display_to_original_map: indices_to_process.add(display_to_original_map[index])
                    else: error_message = f"Некоректний індекс відображення: {index} (макс. {max_display_index})."
                except ValueError: error_message = f"Некоректний формат індексу: '{part}'."
            if error_message: break
        return indices_to_process, error_message

    def _handle_remove_nan_rows(self, column, index_str):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            indices, error_msg = self._parse_index_range(index_str)
            if error_msg:
                logger.error(f"❌ Помилка парсингу індексів: {error_msg}")
                self.analyze_nan_core(self.widgets_map['nan_column_select'].value, self.widgets_map['nan_limit_input'].value, re_display=True)
                return
            if not indices:
                 logger.warning("⚠️ Не вказано коректних індексів для видалення.")
                 self.analyze_nan_core(self.widgets_map['nan_column_select'].value, self.widgets_map['nan_limit_input'].value, re_display=True)
                 return
            self._save_state()
            rows_before = len(self.df)
            self.df.drop(index=list(indices), inplace=True)
            rows_removed = rows_before - len(self.df)
            self.log_change(f"Видалено {rows_removed} рядків (NaN-аналіз стовпця {column}) за індексами {index_str}.")
            logger.info(f"✅ Видалено **{rows_removed}** рядків з NaN-аналізу ({column}).")
            self.widgets_map['nan_indices_remove'].value = ''
            self.analyze_nan_core(self.widgets_map['nan_column_select'].value, self.widgets_map['nan_limit_input'].value, re_display=True)

    def _handle_replace_nan_values(self, column, edit_column, index_str, new_value):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            indices, error_msg = self._parse_index_range(index_str)
            if error_msg:
                logger.error(f"❌ Помилка парсингу індексів: {error_msg}")
                self.analyze_nan_core(self.widgets_map['nan_column_select'].value, self.widgets_map['nan_limit_input'].value, re_display=True)
                return
            if not indices:
                 logger.warning("⚠️ Не вказано коректних індексів для заміни.")
                 self.analyze_nan_core(self.widgets_map['nan_column_select'].value, self.widgets_map['nan_limit_input'].value, re_display=True)
                 return
            
            list_indices = list(indices)
            original_series_subset = self.df.loc[list_indices, edit_column].astype(str).copy()
            
            self._save_state()
            
            self.df.loc[list_indices, edit_column] = new_value
            
            changed_count = (original_series_subset != self.df.loc[list_indices, edit_column].astype(str)).sum()

            self.log_change(f"Замінено значення у {changed_count} комірках стовпця {edit_column} на '{new_value}' (NaN-аналіз) за індексами {index_str}.")
            logger.info(f"✅ Замінено значення у **{changed_count}** комірках ({edit_column}) на **'{new_value}'**.")
            
            self.widgets_map['nan_indices_replace'].value = ''
            self.widgets_map['nan_replace_value'].value = ''
            self.analyze_nan_core(self.widgets_map['nan_column_select'].value, self.widgets_map['nan_limit_input'].value, re_display=True)

    def sort_data_core(self, column, ascending):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            try:
                self._save_state()
                sort_order = 'зростання' if ascending else 'спадання'
                self.df.sort_values(by=column, ascending=ascending, inplace=True, ignore_index=True)
                self.log_change(f"Сортування за стовпцем {column} у порядку {sort_order}.")
                logger.info(f"✅ Дані відсортовано за стовпцем **{column}** ({sort_order}).")
            except Exception as e:
                logger.error(f"❌ Критична помилка при сортуванні: {e}")
                self.df_history.pop()
                self.history_index -= 1

    def flatten_json_column_core(self, column, mode, root_key):
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.df is None or column not in self.df.columns or self.df[column].dtype.kind not in 'OSU':
                logger.error("❌ Дані не завантажено, стовпець не знайдено, або він не є текстовим.")
                return

            rows_before = len(self.df.columns)
            self._save_state()

            try:
                if mode == 'simple_json':
                    logger.info(f"⚙️ Розпаковування простого JSON-об'єкта у стовпці **{column}**...")
                    parsed_series = self.df[column].apply(
                        lambda x: json.loads(str(x).replace('\\n', '').replace('""', '"')) if pd.notna(x) and str(x).strip().startswith(('{', '[')) else None
                    )
                    
                    df_normalized = pd.json_normalize(
                        parsed_series.dropna()
                    ).set_index(parsed_series.dropna().index)

                    prefix = f'{column}_'
                    df_normalized.columns = [f'{prefix}{col}' for col in df_normalized.columns]
                    
                    original_index_name = self.df.index.name
                    self.df.index.name = None
                    self.df = self.df.join(df_normalized)
                    self.df.index.name = original_index_name 

                    
                elif mode == 'ga4_params':
                    logger.info(f"⚙️ Розпаковування GA4 event_params/user_properties ({root_key}) у стовпці **{column}**...")
                    
                    data_dicts = self.df[column].apply(
                        lambda x: self._flatten_ga4_event_params(x, root_key)
                    )
                    
                    df_normalized = pd.DataFrame(
                        data_dicts.tolist(), 
                        index=data_dicts.index
                    )

                    original_index_name = self.df.index.name
                    self.df.index.name = None
                    self.df = self.df.join(df_normalized)
                    self.df.index.name = original_index_name 
                    
                else:
                    logger.error("❌ Невідомий режим розпакування JSON.")
                    self.df_history.pop()
                    self.history_index -= 1
                    return
                
                new_columns_count = len(self.df.columns) - rows_before
                if new_columns_count > 0:
                    self.log_change(f"Розпаковано JSON у стовпці {column} ({mode}). Додано {new_columns_count} нових стовпців.")
                    logger.info(f"✅ У стовпці **{column}** виконано розпакування JSON. Додано **{new_columns_count}** нових стовпців.")
                else:
                    logger.warning(f"⚠️ Розпакування JSON у стовпці {column} не призвело до додавання нових стовпців (можливо, дані були пустими або неправильного формату).")
                    self.df_history.pop()
                    self.history_index -= 1
                    
            except Exception as e:
                logger.error(f"❌ Критична помилка під час розпакування JSON у стовпці {column}: {e}")
                self.df_history.pop()
                self.history_index -= 1

    def _handle_save_file(self, b):
        file_name = self.widgets_map['save_filename_input'].value.strip()
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if self.df is None:
                logger.error("❌ Датасет не завантажено.")
                return
            if not file_name:
                logger.error("❌ Ім'я файлу не може бути порожнім.")
                return
            full_path = os.path.join(self.folder_path, file_name)
            file_extension = os.path.splitext(file_name)[1].lower()
            try:
                if file_extension == '.csv':
                    self.df.to_csv(full_path, index=False, encoding='utf-8')
                    logger.info(f"✅ Файл успішно збережено як **{file_name}** у папці: **{self.folder_path}**")
                elif file_extension in ['.xls', '.xlsx']:
                    sheet_name = self.current_sheet_name if isinstance(self.current_sheet_name, str) else 'Cleaned_Data'
                    with pd.ExcelWriter(full_path, engine='xlsxwriter') as writer:
                        self.df.to_excel(writer, sheet_name=sheet_name, index=False)
                    logger.info(f"✅ Файл успішно збережено як **{file_name}** (Лист: {sheet_name}) у папці: **{self.folder_path}**")
                else:
                    logger.error("❌ Непідтримуване розширення файлу. Використовуйте .csv або .xlsx.")
            except Exception as e:
                logger.error(f"❌ Помилка при збереженні файлу: {e}")

# ==============================================================================
    
    def _update_action_ui(self, change):
        action_value = self.widgets_map['action_dropdown'].value
        with self.widgets_map['dynamic_controls']:
            clear_output(wait=True)
            if action_value is None or self.df is None: return
            
            columns = self._get_column_options()
            initial_column = self.ui_state.get('column_select')
            if initial_column not in columns: initial_column = columns[0] if columns else None
            
            column_select = widgets.Dropdown(options=columns, description='Стовпець:', value=initial_column, layout={'width': 'auto'})
            apply_btn = widgets.Button(description='Застосувати', button_style='primary', layout={'width': 'auto'})
            
            if action_value == 'drop_duplicates':
                duplicate_count = self.df.duplicated().sum()
                info_html = widgets.HTML(f"**ℹ️ Знайдено дублікатів: {duplicate_count}** ({duplicate_count / len(self.df) * 100:.2f}%)")
                
                keep_radio = widgets.RadioButtons(options={'Перший дублікат': 'first', 'Останній дублікат': 'last', 'Жоден (видалити всі)': False}, value='first', description='Залишити:')
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, keep=keep_radio.value))
                
                display(widgets.VBox([info_html, keep_radio, apply_btn]))

            elif action_value == 'fillna_zero':
                numeric_cols = self._get_column_options(dtype_include='numeric')
                num_cols_options = ['--- Всі числові стовпці ---'] + numeric_cols
                col_select_num = widgets.Dropdown(options=num_cols_options, description='Стовпець:', value=num_cols_options[0] if num_cols_options else None, layout={'width': 'auto'})
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=col_select_num.value))
                display(widgets.VBox([col_select_num, apply_btn]))

            elif action_value == 'transliterate':
                mode_radio = widgets.RadioButtons(options={'Повна (ру/укр на лат)': 'full', 'Схожі ру/укр на лат': 'cyr_to_lat_lookalikes', 'Схожі лат на ру/укр': 'lat_to_cyr_lookalikes'}, value='full', description='Режим:')
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select.value, mode=mode_radio.value))
                display(widgets.VBox([column_select, mode_radio, apply_btn]))

            elif action_value == 'normalize_spaces':
                text_cols = self._get_column_options(include_all=True, dtype_include='text')
                column_select_text = widgets.Dropdown(options=text_cols, description='Стовпець:', value=text_cols[0] if text_cols else None, layout={'width': 'auto'})
                
                info_html = widgets.HTML("")
                
                def update_space_summary(change):
                    col = change.get('new') if isinstance(change, dict) and 'new' in change else change.new
                    
                    cols_to_process = []
                    
                    if col == '--- Всі текстові стовпці ---':
                        cols_to_process = self._get_column_options(dtype_include='text')
                    elif col in self.df.columns and self.df[col].dtype.kind in 'OSU':
                        cols_to_process = [col]
                    
                    total_changed_count = 0
                    if cols_to_process:
                        for c in cols_to_process:
                             original_series = self.df[c].astype(str).copy()
                             cleaned_series = self.df[c].astype(str).str.replace(r'\\s+', ' ', regex=True).str.strip()
                             changed_count = (original_series != cleaned_series).sum()
                             total_changed_count += changed_count
                             
                    info_html.value = f"**ℹ️ Буде змінено комірок: {total_changed_count}**"
                
                column_select_text.observe(update_space_summary, names='value')
                update_space_summary({'new': column_select_text.value}) 

                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select_text.value))
                display(widgets.VBox([column_select_text, info_html, apply_btn]))

            elif action_value == 'normalize_case':
                case_type = widgets.RadioButtons(options=['Title Case (Кожне слово з великої)', 'UPPER CASE (ВСІ ВЕЛИКІ)', 'lower case (всі малі)', 'Sentence Case (Перший символ великий)'], value='Title Case (Кожне слово з великої)', description='Тип:')
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select.value, case_type=case_type.value))
                display(widgets.VBox([column_select, case_type, apply_btn]))

            elif action_value == 'to_datetime':
                
                def create_format_builder(title, part_list, num_parts):
                    dropdowns = []
                    dropdown_layout = widgets.Layout(width='200px') 
                    
                    rows = []
                    for i in range(num_parts):
                        options = self.DATE_ELEMENTS if i % 2 == 0 else self.DATE_SEPARATORS
                        initial_value = self.ui_state['date_transform'][part_list][i] if i < len(self.ui_state['date_transform'][part_list]) else '--- Вибрати ---'
                        
                        dropdown = widgets.Dropdown(options=list(options.keys()), 
                                                    value=initial_value, 
                                                    description=f'Частина {i+1}:', 
                                                    layout=dropdown_layout)
                        dropdowns.append(dropdown)
                    
                    row_parts = []
                    parts_per_row = 3 if num_parts in [3, 6] else 4 
                    for i in range(0, num_parts, parts_per_row):
                         row_parts.append(widgets.HBox(dropdowns[i:i + parts_per_row]))
                    rows = widgets.VBox(row_parts)

                    def on_change(change):
                         current_list = [d.value for d in dropdowns]
                         self.ui_state['date_transform'][part_list] = current_list
                         
                    for dd in dropdowns: dd.observe(on_change, names='value')
                    
                    return widgets.VBox([widgets.HTML(f"<h4>{title} ({num_parts} частин):</h4>"), rows], 
                                         layout=widgets.Layout(border='1px solid lightgray', padding='5px', margin='0 0 5px 0'))
                
                date_mode_options = {
                    '1. Стандартна (Pandas Auto, для чистих/змішаних дат)': 'soft_convert', 
                    '2. М\'яке Очищення + Авто (для брудних дат, напр. "22.03.22 р.")': 'soft_clean_infer', 
                    '3. Жорстка (Вилучення за Вхідним Форматом, для брудних дат)': 'hard_convert_pattern'
                }
                date_mode_radio = widgets.RadioButtons(
                    options=date_mode_options,
                    value=self.ui_state['date_transform']['date_mode'], 
                    description='Режим конвертації:',
                    layout=widgets.Layout(width='auto')
                )
                
                source_box = create_format_builder("Вхідний Формат (Стовпець):", 'source_parts', 6)
                target_date_format_builder = create_format_builder("Вихідний Формат (Дата):", 'target_parts', 6)
                
                keep_time_checkbox = widgets.Checkbox(description='Зберігати Час (HH:MM:SS + Мілісекунди)', 
                                                      value=self.ui_state['date_transform']['keep_time'], 
                                                      layout=widgets.Layout(width='auto')) 

                time_format_builder_box = create_format_builder("Вихідний Формат (Час):", 'target_time_parts', 7)
                
                time_format_builder_box.layout.display = 'block' if keep_time_checkbox.value else 'none'
                
                def toggle_time_builder(change):
                    is_checked = change.get('new', keep_time_checkbox.value)
                    time_format_builder_box.layout.display = 'block' if is_checked else 'none'
                    self.ui_state['date_transform']['keep_time'] = is_checked 

                keep_time_checkbox.observe(toggle_time_builder, names='value')

                target_options = {'Рік': 'year', 'Місяць': 'month', 'День': 'day', 'Година': 'hour', 'Хвилина': 'minute', 'Секунда': 'second'}
                checkbox_layout = widgets.Layout(width='auto', margin='0 10px 0 0') 
                target_checkboxes = {name: widgets.Checkbox(description=name, 
                                                            value=True if name in ['Рік', 'Місяць', 'День'] else False, 
                                                            layout=checkbox_layout) 
                                     for name in target_options.keys()}
                                     
                date_row = widgets.HBox([target_checkboxes[name] for name in ['Рік', 'Місяць', 'День']], layout=widgets.Layout(width='400px'))
                time_row = widgets.HBox([target_checkboxes[name] for name in ['Година', 'Хвилина', 'Секунда']], layout=widgets.Layout(width='400px'))
                target_checkbox_box = widgets.VBox([date_row, time_row])
                target_checkbox_container = widgets.VBox([widgets.HTML("<h4>Частини для виділення:</h4>"), target_checkbox_box])

                output_radio = widgets.RadioButtons(options={'Конвертувати та Змінити Формат': 'replace_format', 'Створити нові стовпці': 'create_new'}, value='replace_format', description='Дія:')
                dynamic_container_output = widgets.VBox([]) 
                
                info_html = widgets.HTML("")

                def update_date_ui(change):
                    date_mode = date_mode_radio.value
                    output_mode = output_radio.value

                    if date_mode == 'soft_clean_infer' or date_mode == 'soft_convert':
                         source_box.layout.display = 'none'
                         info_text = "ℹ️ Вхідний формат не потрібен для цього режиму."
                    else:
                         source_box.layout.display = 'block'
                         info_text = "ℹ️ Вхідний формат використовується як маска для вилучення."
                    info_html.value = info_text
                    
                    if output_mode == 'replace_format':
                        dynamic_container_output.children = [target_date_format_builder, keep_time_checkbox, time_format_builder_box]
                        toggle_time_builder({'new': keep_time_checkbox.value})
                    else:
                        dynamic_container_output.children = [target_checkbox_container]

                output_radio.observe(update_date_ui, names='value')
                date_mode_radio.observe(update_date_ui, names='value')
                update_date_ui({'new': output_radio.value, 'owner': output_radio}) 

                def handle_apply(b):
                    src_parts_raw = self.ui_state['date_transform']['source_parts']
                    
                    if output_radio.value == 'create_new':
                        tgt_parts = [name for name, cb in target_checkboxes.items() if cb.value]
                        self._handle_action_apply(action_value, column=column_select.value, source_parts=src_parts_raw, target_parts=tgt_parts, output_mode='create_new', keep_time=None, date_mode=date_mode_radio.value)
                    else:

                        keep_time_val = keep_time_checkbox.value
                        
                        final_target_parts = [p for p in self.ui_state['date_transform']['target_parts'] if p != '--- Вибрати ---']
                        
                        if keep_time_val:
                            time_parts = [p for p in self.ui_state['date_transform']['target_time_parts'] if p != '--- Вибрати ---']
                            
                            if final_target_parts and final_target_parts[-1] not in self.DATE_SEPARATORS and any(p in ['Година (HH)', 'Хвилина (MM)', 'Секунда (SS)', 'Мікросекунди (f)'] for p in self.ui_state['date_transform']['target_time_parts']):
                                final_target_parts.append('Пробіл (Space)') 
                            
                            final_target_parts.extend(time_parts)

                        target_format_parts_codes = [self.DATE_ELEMENTS.get(p) or self.DATE_SEPARATORS.get(p) for p in final_target_parts if p != '--- Вибрати ---']
                        target_format = ''.join(target_format_parts_codes)
                        
                        has_time_elements = any(code in target_format_parts_codes for code in ['%H', '%M', '%S', '%f'])
                        
                        if keep_time_val and not has_time_elements:
                             logger.warning("⚠️ ВИХІДНИЙ ФОРМАТ не містить компонентів часу (Година, Хвилина, Секунда, Мікросекунди), хоча опція 'Зберігати Час' увімкнена. Час буде втрачено при форматуванні.")

                        self._handle_action_apply(action_value, column=column_select.value, source_parts=src_parts_raw, target_parts=final_target_parts, output_mode='replace_format', keep_time=keep_time_val, date_mode=date_mode_radio.value)

                apply_btn.on_click(handle_apply)
                
                display(widgets.VBox([column_select, date_mode_radio, info_html, output_radio, source_box, dynamic_container_output, apply_btn]))

            elif action_value == 'data_management':
                
                mode_radio = widgets.RadioButtons(
                    options={
                        'Перейменувати стовпець': 'rename', 
                        'Видалити стовпець': 'column', 
                        'Видалити рядки з NaN': 'rows_with_nan', 
                        'Пошук/Заміна в Назвах Стовпців': 'rename_bulk',
                        'Назви: Title Case (Кожне слово з великої)': 'col_title_case',
                        'Назви: lower case (всі малі)': 'col_lower_case',
                        'Назви: Sentence Case (Перший символ великий)': 'col_sentence_case'
                    }, 
                    value='rename', 
                    description='Дія:',
                    layout=widgets.Layout(width='450px') 
                ) 
                
                rename_input = widgets.Text(description='Нове ім\'я:', placeholder='New_Column_Name')
                find_col_name = widgets.Text(description='Знайти підрядок:', placeholder='old_prefix_or_suffix')
                replace_col_name = widgets.Text(description='Замінити на:', placeholder='new_string')

                def update_management_ui(change):
                    mode = change.get('new') if isinstance(change, dict) else change.new
                    controls = [mode_radio]
                    
                    if mode in ['rename', 'column']:
                        controls.append(column_select)
                    if mode == 'rename':
                        controls.append(rename_input)
                    if mode == 'rename_bulk': 
                        controls.extend([find_col_name, replace_col_name])
                        
                    controls.append(apply_btn)
                    management_container.children = controls

                mode_radio.observe(update_management_ui, names='value')
                
                def handle_management_apply(b):
                    mode = mode_radio.value
                    col = column_select.value
                    
                    kwargs = {'mode': mode, 'column': col, 'new_name': None, 'find_value': None, 'replace_value': None, 'case_type': None}

                    if mode == 'rename':
                        kwargs['new_name'] = rename_input.value
                    elif mode in ['column', 'rows_with_nan']:
                        pass 
                    elif mode == 'rename_bulk': 
                        kwargs['find_value'] = find_col_name.value
                        kwargs['replace_value'] = replace_col_name.value
                        kwargs['column'] = None
                    elif mode == 'col_title_case':
                        kwargs['case_type'] = 'title'
                        kwargs['mode'] = 'normalize_case_col' 
                        kwargs['column'] = None
                    elif mode == 'col_lower_case':
                        kwargs['case_type'] = 'lower'
                        kwargs['mode'] = 'normalize_case_col'
                        kwargs['column'] = None
                    elif mode == 'col_sentence_case': 
                        kwargs['case_type'] = 'sentence'
                        kwargs['mode'] = 'normalize_case_col'
                        kwargs['column'] = None
                
                    self._handle_action_apply(action_value, **kwargs)
                
                apply_btn.on_click(handle_management_apply)
                management_container = widgets.VBox()
                update_management_ui({'new': mode_radio.value})
                display(management_container)

            elif action_value == 'fill_missing_stats':
                num_cols = self._get_column_options(dtype_include='numeric')
                text_cols = self._get_column_options(dtype_include='text')
                all_cols = num_cols + text_cols
                
                column_select_stats = widgets.Dropdown(options=all_cols, description='Стовпець:', value=all_cols[0] if all_cols else None, layout={'width': 'auto'})
                
                method_radio = widgets.RadioButtons(options=['Медіана (для чисел)', 'Середнє (для чисел)', 'Мінімум (для чисел)', 'Максимум (для чисел)', 'Мода (для всіх типів)'], 
                                                    value='Мода (для всіх типів)', description='Метод:')
                
                def get_method_value(label):
                    if 'Медіана' in label: return 'median'
                    if 'Середнє' in label: return 'mean'
                    if 'Мінімум' in label: return 'min'
                    if 'Максимум' in label: return 'max'
                    if 'Мода' in label: return 'mode'
                    return None

                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select_stats.value, method=get_method_value(method_radio.value)))
                
                display(widgets.VBox([column_select_stats, method_radio, apply_btn]))

            elif action_value == 'check_unique':
                apply_btn.description = "Показати Унікальні / Статистику + Графік"
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select.value))
                display(widgets.VBox([column_select, apply_btn]))
            
            elif action_value == 'find_replace':
                state = self.ui_state['find_replace']
                unique_options = ['nan/Пропущене'] + [str(v) for v in self.df[column_select.value].unique() if v is not np.nan] if column_select.value in self.df.columns else ['---']
                unique_value_select = widgets.Dropdown(options=unique_options, description='Знайти:', layout={'width': 'auto'})
                find_input = widgets.Text(description='Знайти:', placeholder='Текст або Regex', value=state['find_input'], layout={'width': 'auto'})
                replace_input = widgets.Text(description='Замінити на:', value=state['replace_input'], layout={'width': 'auto'})
                case_sensitive_check = widgets.Checkbox(value=state['case_sensitive_check'], description='Чутливий до регістру')
                mode_radio = widgets.RadioButtons(options={'Унікальне значення': 'unique', 'Підрядок / Regex': 'substring'}, value=state['mode_radio'], description='Режим:')
                regex_check = widgets.Checkbox(value=state['regex_check'], description='Використовувати Regex')
                whole_cell_radio = widgets.RadioButtons(options={'Повний збіг': 'whole', 'Частина клітинки': 'part'}, value=state['whole_cell_radio'], description='Збіг:')
                substring_options = widgets.VBox([whole_cell_radio, regex_check])
                find_widget_container = widgets.VBox()

                def update_unique_options(change):
                    col = change.get('new', column_select.value)
                    if col and col in self.df.columns: 
                        unique_options = [str(v) for v in self.df[col].unique() if v is not np.nan]
                        if self.df[col].isnull().any(): unique_options.insert(0, 'nan/Пропущене')
                        unique_value_select.options = unique_options
                        unique_value_select.value = unique_options[0] if unique_options else None

                def toggle_find_widget(change):
                    is_substring = change.get('new', mode_radio.value) == 'substring'
                    substring_options.layout.display = 'flex' if is_substring else 'none'
                    find_widget_container.children = [find_input] if is_substring else [unique_value_select]
                
                column_select.observe(update_unique_options, names='value')
                mode_radio.observe(toggle_find_widget, names='value')
                update_unique_options({})
                toggle_find_widget({})

                def get_find_value(): 
                    val = find_input.value if mode_radio.value == 'substring' else unique_value_select.value
                    return np.nan if val == 'nan/Пропущене' else val

                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select.value, find_value=get_find_value(), replace_value=replace_input.value, case_sensitive=case_sensitive_check.value, mode=mode_radio.value, use_regex=regex_check.value, whole_cell_match=(whole_cell_radio.value == 'whole')))
                display(widgets.VBox([column_select, find_widget_container, replace_input, case_sensitive_check, mode_radio, substring_options, apply_btn]))

            elif action_value == 'filter_data':
                condition_options_num = {'< (менше)': 'less_than', '> (більше)': 'greater_than', '= (дорівнює)': 'equal', '≠ (не дорівнює)': 'not_equal', 'NaN': 'is_nan', 'Не NaN': 'is_not_nan'}
                condition_options_text = {'Містить': 'contains', 'Не містить': 'not_contains', '= (дорівнює)': 'equal', '≠ (не дорівнює)': 'not_equal', 'NaN': 'is_nan', 'Не NaN': 'is_not_nan'}
                condition_select = widgets.Dropdown(options=['--- Виберіть Умову ---'], description='Умова:', layout={'width': 'auto'})
                value_input = widgets.Text(description='Значення:', placeholder='Значення для порівняння')
                value_input.layout.display = 'block'
                
                def update_filter_options(change):
                    col = change.get('new') if isinstance(change, dict) else change.new
                    
                    is_numeric = pd.api.types.is_numeric_dtype(self.df[col]) if col in self.df.columns else False
                    
                    options = condition_options_num if is_numeric else condition_options_text
                    options_list = ['--- Виберіть Умову ---'] + list(options.keys())
                    
                    current_value = condition_select.value
                    condition_select.options = options_list
                    condition_select.value = current_value if current_value in options_list else '--- Виберіть Умову ---'
                    
                def update_value_visibility(change):
                    condition = condition_select.value
                    if condition in ['NaN', 'Не NaN', '--- Виберіть Умову ---']:
                        value_input.layout.display = 'none'
                        value_input.value = ''
                    else:
                        value_input.layout.display = 'block'
                
                column_select.observe(update_filter_options, names='value')
                condition_select.observe(update_value_visibility, names='value')
                update_filter_options({'new': column_select.value})

                def get_condition_value(key): return condition_options_num.get(key) or condition_options_text.get(key)

                apply_btn.description = "Фільтрувати та Видалити Не Відповідні"
                apply_btn.button_style = 'danger'
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select.value, condition_type=get_condition_value(condition_select.value), value=value_input.value))
                display(widgets.VBox([column_select, condition_select, value_input, apply_btn]))

            elif action_value == 'analyze_nan':
                analyze_btn = widgets.Button(description='Показати Рядки з NaN', button_style='warning', layout={'width': 'auto'})
                analyze_btn.on_click(lambda b: self._handle_action_apply(action_value, column=self.widgets_map['nan_column_select'].value, limit=self.widgets_map['nan_limit_input'].value))
                
                self.widgets_map['nan_column_select'].options = columns
                self.widgets_map['nan_column_select'].value = initial_column
                
                self.widgets_map['nan_column_select_edit'].options = columns
                self.widgets_map['nan_column_select_edit'].value = initial_column
                
                nan_removal_panel = widgets.VBox([
                    widgets.HTML("<h4>Видалення рядків:</h4>"),
                    self.widgets_map['nan_indices_remove'],
                    self.widgets_map['nan_remove_btn']
                ])
                nan_replace_panel = widgets.VBox([
                    widgets.HTML("<h4>Заміна значень (для обраних індексів):</h4>"), 
                    self.widgets_map['nan_column_select_edit'],
                    self.widgets_map['nan_indices_replace'],
                    self.widgets_map['nan_replace_value'],
                    self.widgets_map['nan_replace_btn']
                ])
                
                display(widgets.VBox([self.widgets_map['nan_column_select'], self.widgets_map['nan_limit_input'], analyze_btn, nan_removal_panel, nan_replace_panel]))

            elif action_value == 'sort_data':
                sort_order_radio = widgets.RadioButtons(options={'За зростанням (A-Z/0-9)': True, 'За спаданням (Z-A/9-0)': False}, value=True, description='Порядок:')
                apply_btn.description = "Сортувати"
                apply_btn.on_click(lambda b: self._handle_action_apply(action_value, column=column_select.value, ascending=sort_order_radio.value))
                display(widgets.VBox([column_select, sort_order_radio, apply_btn]))

            elif action_value == 'save_file':
                current_base_name = os.path.splitext(self.current_file_name)[0] if self.current_file_name else 'cleaned_data'
                current_ext = os.path.splitext(self.current_file_name)[1] if self.current_file_name and os.path.splitext(self.current_file_name)[1] in ['.csv', '.xls', '.xlsx'] else '.xlsx'
                
                suggested_name = f"{current_base_name}_cleaned{current_ext}"
                self.widgets_map['save_filename_input'].value = suggested_name
                
                display(widgets.VBox([self.widgets_map['save_filename_input'], self.widgets_map['save_btn']]))
            
            elif action_value == 'flatten_json_column':
                text_cols = self._get_column_options(dtype_include='text')
                if not text_cols:
                    display(widgets.HTML("<b>⚠️ Текстових стовпців для розпакування JSON не знайдено.</b>"))
                    return
                
                self.widgets_map['json_column_select'].options = text_cols
                if initial_column in text_cols:
                    self.widgets_map['json_column_select'].value = initial_column
                else:
                    self.widgets_map['json_column_select'].value = text_cols[0]
                
                def toggle_root_key(change):
                    if change.get('new') == 'ga4_params':
                        self.widgets_map['json_target_column_input'].layout.display = 'block'
                    else:
                        self.widgets_map['json_target_column_input'].layout.display = 'none'
                
                self.widgets_map['json_flatten_mode'].observe(toggle_root_key, names='value')

                toggle_root_key({'new': self.widgets_map['json_flatten_mode'].value})

                display(widgets.VBox([
                    self.widgets_map['json_column_select'],
                    self.widgets_map['json_flatten_mode'],
                    self.widgets_map['json_target_column_input'],
                    self.widgets_map['json_apply_btn']
                ]))
            
            else:
                pass


    def _handle_action_apply(self, action_name, **kwargs):
        if self.df is None:
            logger.error("❌ Датасет не завантажено.")
            return
        
        if kwargs.get('column') and action_name not in ['drop_duplicates', 'save_file', 'data_management', 'fillna_zero']:
            self.ui_state['column_select'] = kwargs['column']

        action_map = {
            'drop_duplicates': (self.drop_duplicates_core, ['keep']),
            'fillna_zero': (self.fillna_zero_core, ['column']),
            'transliterate': (self.transliterate_core, ['column', 'mode']),
            'normalize_spaces': (self.normalize_spaces_core, ['column']),
            'normalize_case': (self.normalize_case_core, ['column', 'case_type']),
            'to_datetime': (self.to_datetime_core, ['column', 'source_parts', 'target_parts', 'output_mode', 'keep_time', 'date_mode']), 
            'data_management': (lambda mode, column, new_name, find_value, replace_value, case_type: 
                               self.rename_column_core(column, new_name) if mode == 'rename' 
                               else (self.drop_data_core(mode, column) if mode in ['column', 'rows_with_nan'] 
                                     else (self.find_replace_column_names_core(find_value, replace_value) if mode == 'rename_bulk' 
                                           else (self.normalize_column_names_case_core(case_type) if mode == 'normalize_case_col'
                                                 else logger.error("❌ Необхідно вибрати режим."))
                                          )
                                    ), 
                               ['mode', 'column', 'new_name', 'find_value', 'replace_value', 'case_type']),
            'fill_missing_stats': (self.fill_missing_stats_core, ['column', 'method']), 
            'check_unique': (self.generate_unique_values, ['column']),
            'find_replace': (self.find_and_replace_core, ['column', 'find_value', 'replace_value', 'mode', 'use_regex', 'whole_cell_match', 'case_sensitive']),
            'filter_data': (self.filter_data_core, ['column', 'condition_type', 'value']),
            'analyze_nan': (self.analyze_nan_core, ['column', 'limit']),
            'remove_nan_rows': (self._handle_remove_nan_rows, ['column', 'index_str']),
            'replace_nan_values': (self._handle_replace_nan_values, ['column', 'edit_column', 'index_str', 'new_value']),
            'sort_data': (self.sort_data_core, ['column', 'ascending']),
            'flatten_json_column': (self.flatten_json_column_core, ['column', 'mode', 'root_key']),
        }
        
        if action_name in action_map:
            func, arg_keys = action_map[action_name]
            args = {key: kwargs.get(key) for key in arg_keys}
            
            try:
                func(**args)
                
                if self.widgets_map['action_dropdown'].value == action_name:
                    self._update_action_ui(None)
                
                if action_name not in ['analyze_nan', 'remove_nan_rows', 'replace_nan_values', 'save_file', 'check_unique']:
                     self._display_summary() 
                
            except Exception as e:
                logger.error(f"❌ Критична помилка виконання дії '{action_name}': {e}")
                if self.history_index >= 0 and action_name not in ['analyze_nan', 'remove_nan_rows', 'replace_nan_values', 'save_file']: 
                    self.df = self.df_history[self.history_index].copy()
                    logger.info("↩️ Стан відновлено після помилки.")
                
                self._display_summary()
                self._update_action_ui(None)
                
    def _handle_folder_submit(self, b):
        self.folder_path = self.widgets_map['folder_input'].value
        with self.widgets_map['main_output']:
            clear_output(wait=True)
            supported_files = self._get_all_files()
            if not os.path.isdir(self.folder_path):
                logger.error(f"❌ Папка не знайдена: {self.folder_path}")
                self._show_post_load_actions(False)
                return
            
            logger.info(f"📁 Робоча папка встановлена: {self.folder_path}")
            
            self.widgets_map['file_dropdown'].options = ['--- Вибрати Файл ---'] + supported_files
            
            if not supported_files:
                logger.warning("⚠️ У папці не знайдено файлів .csv, .xls або .xlsx.")
                self._show_post_load_actions(False)
            else:
                if len(supported_files) > 0:
                     self.widgets_map['file_dropdown'].value = supported_files[0]
                self._show_post_load_actions(True)

    def _handle_file_select(self, change):
        file_name = change.new
        if file_name and file_name != '--- Вибрати Файл ---':
            self.current_file_name = file_name
            ext = os.path.splitext(file_name)[1].lower()
            
            self.widgets_map['sheet_dropdown'].layout.display = 'none'
            self.widgets_map['encoding_input'].layout.display = 'none'
            self.widgets_map['encoding_help'].layout.display = 'none'

            if ext in ['.xls', '.xlsx']:
                self.widgets_map['sheet_dropdown'].layout.display = 'block'
                full_path = os.path.join(self.folder_path, file_name)
                try:
                    xl = pd.ExcelFile(full_path)
                    sheet_names = xl.sheet_names
                    self.widgets_map['sheet_dropdown'].options = sheet_names
                    self.widgets_map['sheet_dropdown'].value = sheet_names[0] if sheet_names else None
                except Exception as e:
                    with self.widgets_map['main_output']:
                        logger.error(f"❌ Помилка читання Excel: {e}")
            elif ext == '.csv':
                 self.widgets_map['encoding_input'].layout.display = 'block'
                 self.widgets_map['encoding_help'].layout.display = 'block'

            if ext == '.csv':
                self.widgets_map['load_settings_container'].children = [
                    self.widgets_map['header_input'], 
                    self.widgets_map['encoding_input'], 
                    self.widgets_map['encoding_help'],
                    self.widgets_map['load_file_btn']
                ]
            elif ext in ['.xls', '.xlsx']:
                self.widgets_map['load_settings_container'].children = [
                    self.widgets_map['sheet_dropdown'], 
                    self.widgets_map['header_input'], 
                    self.widgets_map['load_file_btn']
                ]
            else:
                 self.widgets_map['load_settings_container'].children = []
        else:
             self.widgets_map['load_settings_container'].children = []

    def _handle_file_load(self, b):
        file_name = self.widgets_map['file_dropdown'].value
        header_row = self.widgets_map['header_input'].value
        encoding = self.widgets_map['encoding_input'].value
        
        with self.widgets_map['results_output']:
            clear_output(wait=True)
            if not file_name or file_name == '--- Вибрати Файл ---':
                logger.error("❌ Будь ласка, виберіть файл.")
                return

            full_path = os.path.join(self.folder_path, file_name)
            ext = os.path.splitext(file_name)[1].lower()
            self.df = None
            
            try:
                if ext == '.csv':
                    self.df = pd.read_csv(full_path, header=header_row if header_row >= 0 else None, encoding=encoding)
                    logger.info(f"✅ Файл **{file_name}** успішно завантажено. (Header: {header_row}, Encoding: {encoding})")
                    self.current_sheet_name = None
                elif ext in ['.xls', '.xlsx']:
                    sheet_name = self.widgets_map['sheet_dropdown'].value
                    self.df = pd.read_excel(full_path, sheet_name=sheet_name, header=header_row if header_row >= 0 else None)
                    logger.info(f"✅ Файл **{file_name}** (Лист: **{sheet_name}**) успішно завантажено. (Header: {header_row})")
                    self.current_sheet_name = sheet_name
                else:
                    logger.error("❌ Непідтримуване розширення файлу.")
                    return
                
                self._save_state()
                self._display_summary()
                self.widgets_map['action_panel'].layout.display = 'block' 
                self.widgets_map['summary_accordion'].layout.display = 'block'
                self.widgets_map['action_dropdown'].value = 'drop_duplicates'

            except Exception as e:
                logger.error(f"❌ Помилка при завантаженні файлу: {e}")
                
    def _show_post_load_actions(self, show):
        display_style = 'block' if show else 'none'
        self.widgets_map['file_dropdown'].layout.display = display_style
        self.widgets_map['load_settings_container'].layout.display = display_style
        
        if show:
            self.widgets_map['file_output'].children = [
                self.widgets_map['file_dropdown'], 
                self.widgets_map['load_settings_container']
            ]
        else:
             self.widgets_map['file_output'].children = []
             
        if not show or self.df is None:
            self.widgets_map['action_panel'].layout.display = 'none' 
            self.widgets_map['summary_accordion'].layout.display = 'none'
        
    def interactive_main(self):
        logger.info("--- Data Cleaner App Started ---")
        
        folder_setup = widgets.VBox([
            widgets.HBox([self.widgets_map['folder_input'], self.widgets_map['submit_folder_btn']]),
            self.widgets_map['main_output']
        ])
        
        app_layout = widgets.VBox([
            folder_setup,
            self.widgets_map['file_output'],
            self.widgets_map['action_panel'], 
            self.widgets_map['results_output'],
            self.widgets_map['summary_accordion']
        ])
        
        display(app_layout)
        
        self._show_post_load_actions(False)

In [4]:
app = DataCleanerApp()
app.interactive_main()

16:21:56 | INFO: --- Data Cleaner App Started ---
